In [1]:
import os, wandb

# W&B API for CI/CD

The followin are examples of operations you can perform with the `wandb` python client that might be relevant for CI/CD.

Pre-requisite: the environment variable `WANDB_API_KEY`

In [2]:
assert os.getenv("WANDB_API_KEY"), "You must set the WANDB_API_KEY environment variable"

## Getting metrics tied to the current production model

We can ge the production model from **registry**, then get the metrics via lineage from the associated run.

In [5]:
run = wandb.init(job_type="CI/CD")
path = "marioparreno/model-registry/"
model_name = "Neurovias"
version= "v1"

artifact = run.use_artifact(
    f"{path}{model_name}:{version}",
    type="model"
)

wandb: Currently logged in as: marioparreno. Use `wandb login --relogin` to force relogin


Get the run that created the model:

In [6]:
run = artifact.logged_by()

In [ ]:
def get_auc_metrics(run):
    return {k:v for k , v in run.summary.items() if "AUC" in k}

In [11]:
get_auc_metrics(run)

{'test/AUC_BOH': 0.920010055304173,
 'test/AUC_GLT': 0.915335622232174,
 'test/AUC_MTA': 0.9909969257795346,
 'test/AUC_AHR': 0.9985035109934384,
 'test/AUC_EXS': 0.9949795918367348,
 'test/AUC_GCO': 0.960379013862081,
 'test/AUC_GRB': 0.9222994873003711,
 'test/AUC_GRR': 0.9978858350951374,
 'test/AUC_PAR': 0.94978256367778}

## Getting metrics for a run ID

This is useful if you want to get metrics for a particular experiment ID.

In [12]:
run_path = "marioparreno/Neurovias/gi63l4cc"
api = wandb.Api()
run = api.run(run_path)

In [13]:
get_auc_metrics(run)

{'test/AUC_BOH': 0.9112116641528406,
 'test/AUC_GRR': 0.9897508962220792,
 'test/AUC_AHR': 0.9992517554967192,
 'test/AUC_GCO': 0.954465695736094,
 'test/AUC_PAR': 0.9484779457444608,
 'test/AUC_GRB': 0.9266898461901114,
 'test/AUC_GLT': 0.9115539632781012}

## Getting metrics for a run/s with a tag

It's better to use the Mongo API instead of downloading and iterating through each run if possible

The use case for this is if you wanted to compare a particular run with another run that you have tagged, such a "baseline" or "candidate"

In [19]:
api = wandb.Api()

project = "marioparreno/Neurovias"
tags = ["baseline"]

candidate_runs = api.runs(
    project,
    {"tags": {"$in": tags}}  # this is the Mongo Client
)

In [20]:
assert len(candidate_runs) == 1, "There should be only one baseline run"

In [21]:
for run in candidate_runs:
    print(run.name)

peachy-cosmos-196


In [22]:
candidate_runs[0].summary

{'test/AUC_GCO': 0.9640463239164766, 'val/f1_macro': 0.7894231081008911, 'test/Analisis': {'_latest_artifact_path': 'wandb-client-artifact://auourfmhohj7uilpln3w5z90j0r48jxquvk5i08gv258sf71ar0pfp48mhdu0s3kw7jdhvpe21p0y4frqppzeijznwgt61cbk9pu03qepli9526tjj2za0ae1gdc41gf:latest/test/Analisis.table.json', 'path': 'media/table/test/Analisis_100_092a9064457eec2d73c7.table.json', 'size': 40015, '_type': 'table-file', 'ncols': 16, 'nrows': 549, 'sha256': '092a9064457eec2d73c7bfc8fac5eefd7c8e31e9f8bc14353a3f1101ff6741cc', 'artifact_path': 'wandb-client-artifact://auourfmhohj7uilpln3w5z90j0r48jxquvk5i08gv258sf71ar0pfp48mhdu0s3kw7jdhvpe21p0y4frqppzeijznwgt61cbk9pu03qepli9526tjj2za0ae1gdc41gf:latest/test/Analisis.table.json'}, 'test/AUC_GLT': 0.906180026869682, 'test/AUC_GRR': 0.9984373563746668, 'train/f1_macro': 0.923873245716095, '_step': 100, '_wandb': {'runtime': 8226}, '_runtime': 8226.600408792496, 'test/ROC': {'nrows': 347, 'sha256': 'e7f469c446b5ce3e619b9d3733b87393b1e3a5e98fc28dc4d8c5f4

In [16]:
get_auc_metrics(run)

{}

## Promote a model to the registry

This is something you could do as part of a CI/CD process

In [22]:
path = "marioparreno/model-registry/"
model_name = "Neurovias"
path = path + model_name

In [30]:
run_path = "marioparreno/Neurovias/boqfgfve"  # this is the run id
api = wandb.Api()
run = api.run(run_path)

Get the model from the run and promote it to the registry

In [31]:
art = [a for a in run.logged_artifacts() if a.type == 'model']

In [33]:
if art:
    assert len(art) == 1, "More than 1 artifact of type model!"
    art[0].link(path, aliases=["production"])

We can see a model in the registry like this:

In [34]:
api = wandb.Api()
versions = api.artifact_versions('model', path)

In [35]:
for v in versions:
    print(f"Model version: {v.version} with tags: {v.aliases}")

Model version: v2 with tags: ['latest', 'production']
Model version: v1 with tags: []
Model version: v0 with tags: ['classification']


## Programatically Reports: Run Comparison

Check the [documentation](https://docs.wandb.ai/guides/reports/create-a-report) of programatically reports

In [36]:
import wandb
import wandb.apis.reports as wr

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [45]:
PROJECT = "Neurovias"
ENTITY = "marioparreno"

In [46]:
report = wr.Report(
    entity=ENTITY,
    project=PROJECT,
    title="Compare Runs",
    description="A demo of comparing runs programatically"
)

We can compare runs like so, and save the report. In Jupyter Notebook, a preview of the report will automatically appear

In [47]:
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "Run Comparison").set_filters_with_python_expr(
            "Name in ['stellar-smoke-224', 'cute-violet-198']"
        )
    ],
    panels=[
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 15})
    ]
)

In [48]:
report.blocks = report.blocks[:1] + [pg] + report.blocks[1:]

In [49]:
report.save()

Report(project='Neurovias', entity='marioparreno', title='Compare Runs', description='A demo of comparing runs programatically', width='readable', blocks=[PanelGrid(runsets=[Runset(entity='marioparreno', project='Neurovias', name='Run Comparison', query='', filters={'$or': [{'$and': [{'displayName': {'$in': ['stellar-smoke-224', 'cute-violet-198']}}]}]}, order=['-CreatedTimestamp'])], panels=[RunComparer(diff_only='split')])])

In [50]:
report.url

'https://wandb.ai/marioparreno/Neurovias/reports/Compare-Runs--Vmlldzo0NTc1MDE5'